In [1]:
import pandas as pd
import numpy as np
import torchvision as tv
import jieba
from gensim import models
from gensim.models import Word2Vec
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
import torch
show = ToPILImage() # 可以把Tensor转成Image，方便可视化
from sklearn.model_selection import train_test_split

In [2]:
'''
data = pd.read_csv('../../data/df_sen_sub/train.csv')
len(data['content_id'].unique()), len(list(data['content_id']))
content_ids = data['content_id'].unique()
subjs = data['subject'].unique()
df = {}
text, subs, sentis = [], [], []
for i in range(len(content_ids)):
    if i % 100 == 0:print(i)
    dd = data[data['content_id'] == content_ids[i]]
    text.append(dd.loc[dd.index.tolist()[0], 'content'])
    temp_subs, temp_senti = [], []
    for sub in subjs:
        temp_subs.append(sub)
        if len(dd[dd['subject'] == sub]) != 0:
            temp_senti.append(str(int(dd[dd['subject'] == sub]['sentiment_value'])))
        else:
            temp_senti.append('2')
    subs.append(','.join(temp_subs))
    sentis.append(','.join(temp_senti))
df['content'] = text
df['subjects'] = subs
df['sentiments'] = sentis
df = pd.DataFrame(df)
df.to_csv('../../data/df_sen_sub/merge_train.csv', index = False)
#data = pd.read_csv('../../data/df_sen_sub/test_public.csv')
'''

"\ndata = pd.read_csv('../../data/df_sen_sub/train.csv')\nlen(data['content_id'].unique()), len(list(data['content_id']))\ncontent_ids = data['content_id'].unique()\nsubjs = data['subject'].unique()\ndf = {}\ntext, subs, sentis = [], [], []\nfor i in range(len(content_ids)):\n    if i % 100 == 0:print(i)\n    dd = data[data['content_id'] == content_ids[i]]\n    text.append(dd.loc[dd.index.tolist()[0], 'content'])\n    temp_subs, temp_senti = [], []\n    for sub in subjs:\n        temp_subs.append(sub)\n        if len(dd[dd['subject'] == sub]) != 0:\n            temp_senti.append(str(int(dd[dd['subject'] == sub]['sentiment_value'])))\n        else:\n            temp_senti.append('2')\n    subs.append(','.join(temp_subs))\n    sentis.append(','.join(temp_senti))\ndf['content'] = text\ndf['subjects'] = subs\ndf['sentiments'] = sentis\ndf = pd.DataFrame(df)\ndf.to_csv('../../data/df_sen_sub/merge_train.csv', index = False)\n#data = pd.read_csv('../../data/df_sen_sub/test_public.csv')\n"

In [3]:
data = pd.read_csv('../../data/df_sen_sub/merge_train.csv')
data = data.fillna('')
subs = list(data['subjects'])
sentiments = list(data['sentiments'])
content = list(data['content'])
test_data = pd.read_csv('../../data/df_sen_sub/test_public.csv')
test_data = test_data.fillna('')
test_content = list(test_data['content'])

In [22]:
with open('../../data/df_sen_sub/char_add_text.txt','w') as f:
    f.write('\n'.join(list(content) + list(test_content)  + list(subs)))
# 迭代器，使用jieba将句子进行分词
class Sentences(object):# 这个类可以根据实际情况重写，我已经将所有的文章进行分句，并整合到了一个文件里面
    def __init__(self, dirname):
        self.dirname = dirname # 句子所在文件，没句句子占一行
        #jieba.load_userdict("wordBase.txt") # 加载词库

    def __iter__(self):
        #for fname in os.listdir(self.dirname):
        for line in open(self.dirname):
                line = line.replace('\n', '')
                if line == '价格,配置,操控,舒适性,油耗,动力,内饰,安全性,空间,外观':
                    yield line.split(',')
                else:
                    yield list(line)

sentences = []
def train_word2vec(folder_path, size=100):
    global sentences
    sentences = Sentences(folder_path) #生成分词后的句子，是一个二维数组

    # size是词向量长度
    # worker是线程数量，建议与物理线程数量一致
    # min_count是指出现次数小于一定程度，就忽略，0表示不忽略
    model = Word2Vec(sentences, size=size, workers=8, min_count=0)

    # 训练结束就将模型保存起来
    model.save("../../data/df_sen_sub/char_add_word2vec_model")

# 生成50维度的词向量模型
train_word2vec("../../data/df_sen_sub/char_add_text.txt",100)

# 测试训练好的词向量模型，使用model[keyWord]即可获取keyword这个词的词向量
model = Word2Vec.load("../../data/df_sen_sub/char_add_word2vec_model")
sentences = list(sentences)


In [24]:
x_vecs = []
x_test_vecs = []  
ind = 0
word2ind = {}
pretrained = []
senti_sen = []
senti_test_sen = []
for i in range(len(sentences) - len(content) + 1):
    if i < len(content):
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_vecs.append(temp)
        senti_sen.append(t_s[:50])
    elif i != len(sentences) - len(content):
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_test_vecs.append(temp)
        senti_test_sen.append(t_s[:50])  
    else:
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1        
x_vecs = np.array(x_vecs)
x_test_vecs = np.array(x_test_vecs)
print(x_vecs.shape, x_test_vecs.shape)

/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:44: DeprecationWarning: Cal

(8290, 50, 100) (2364, 50, 100)


In [25]:
len(word2ind),word2ind['价格']

(2875, 2865)

In [26]:
#va, content_vecs, y
pretrained.append([0.0] * 100)
word2ind['null'] = 2875
map_ = ['-1', '0', '1', '2']
sub_set = {}
for i in range(len(subs)):
    subjs = subs[i].split(',')
    sentis = sentiments[i].split(',')
    for j in range(len(subjs)):
        if subjs[j] not in sub_set:
            sub_set[subjs[j]] = []
        sub_set[subjs[j]].append(map_.index(sentis[j]))

In [27]:
pretrained = np.array(pretrained)
han_x = []
for i in range(len(senti_sen)):
    temp = []
    for j in range(len(senti_sen[i])):
        temp.append(word2ind[senti_sen[i][j]])
    while len(temp) < 50:
        temp.append(2875)

    temp = temp[:50]

    han_x.append(temp)
han_x = np.array(han_x)

han_x_test = []
for i in range(len(senti_test_sen)):
    temp = []
    for j in range(len(senti_test_sen[i])):
        temp.append(word2ind[senti_test_sen[i][j]])
    while len(temp) < 50:
        temp.append(2875)

    temp = temp[:50]

    han_x_test.append(temp)
han_x = np.array(han_x)
han_x_test = np.array(han_x_test)

In [28]:

'''
class Net(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(Net, self).__init__()
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True) 
        #self.fc1   = nn.Linear(3750, 50) 
        #self.fc2   = nn.Linear(200, 50)
        #self.fc3   = nn.Linear(200, 50)
        #self.fc4   = nn.Linear(100, 3)
        self.fctest   = nn.Linear(200, 3)
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, x, aspects): 
        x = self.embeddings(x)#batch,50,100
        #va = self.embeddings(aspects[0])
        #t_va = va.unsqueeze(0).expand(x.shape[0], x.shape[1], 100)
        output, (hn, cn) =  self.bilstm(x.float())#batch,50,200
        H, hN = output, output[:,-1,:]
        #con = torch.cat((t_va, H), 2)
        #t1 = F.max_pool2d(F.relu(con), (1, 4))
        #con = t1.view(x.size()[0], -1)
        #con = F.relu(self.fc1(con))
        #alpha = con.unsqueeze(1)
        #rr = torch.matmul(alpha,H).squeeze(1)
        #yy = torch.cat((F.relu(self.fc2(rr)), F.relu(self.fc3(hN))), 1)
        #x = self.fc4(yy)
        x  = self.fctest(hN)
        return x
    

net = Net(pretrained, len(pretrained))
print(net)
from torch import optim
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)
'''
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(Net, self).__init__()
        #self.rnn1 = nn.RNN(100, 100) 
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True)
        #self.fc1   = nn.Linear(1250, 120) 
        self.fch = nn.Linear(200, 100)
        self.fcv = nn.Linear(100, 100)
        self.fcw = nn.Linear(200, 1)
        self.fcwp = nn.Linear(200,50)
        self.fcwh = nn.Linear(200,50)
        self.fc1   = nn.Linear(3750, 120) 
        self.fc2   = nn.Linear(120, 50)
        self.fc3   = nn.Linear(50, 4)
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, x, aspects): 
        x = self.embeddings(x) #[4,50,100]
        batch_size = x.shape[0]
        N = x.shape[1]
        output, (hn, cn) =  self.bilstm(x.float())  #[4,50,200]
        hN = output[:,-1,:]
        va = self.embeddings(aspects[0]) #[1, 100]
        t_va = torch.matmul(torch.ones((x.shape[0],50, 1)), va)#[4, 50, 100]
        t_va = t_va.reshape((-1, 100)) #[4*50, 100]
        x = output.reshape((-1, 200)) #[4*50, 200]
        x = torch.tanh(torch.cat((self.fch(x), self.fcv(t_va)), 1))
        M = x.reshape((batch_size, N, -1)) #[4, 50, 200]
        alpha = F.softmax(self.fcw(M)).reshape((batch_size, 1, -1))#[4,50]
        rr = torch.matmul(alpha, output).reshape((batch_size, -1)) #[4, 200]
        final = torch.tanh(self.fcwh(hN) + self.fcwp(rr))
        x = self.fc3(final)   
        return x
    


In [30]:
ii = 0
import time
for key in sub_set:
    if ii == 10:
        break
    ii += 1
    net = Net(pretrained, len(pretrained))
    cur_max_acc = 0.0
    print(net)
    from torch import optim
    criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
    optimizer = optim.Adagrad(net.parameters(), lr=0.01)#, momentum=0.9)
    train_x, test_x, train_y, test_y = train_test_split(han_x, sub_set[key], test_size=0.01, random_state=42)
    trainset = [(train_x[i], train_y[i]) for i in range(len(train_x))]
    testset = [(test_x[i], test_y[i]) for i in range(len(test_x))]
    trainloader = torch.utils.data.DataLoader(
                        trainset, 
                        batch_size=4,
                        shuffle=True, 
                        num_workers=2)
    testloader = torch.utils.data.DataLoader(
                        testset, 
                        batch_size=4,
                        shuffle=False, 
                        num_workers=2)
    torch.set_num_threads(8)
    for epoch in range(20):  

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):

            # 输入数据
            inputs, labels = data
            # 梯度清零
            optimizer.zero_grad()
            # forward + backward 
            va = [[word2ind[key]]]
            va = torch.from_numpy(np.array(va))
            outputs = net(inputs, va)
            loss = criterion(outputs, labels)
            loss.backward()   

            # 更新参数 
            optimizer.step()

            # 打印log信息
            # loss 是一个scalar,需要使用loss.item()来获取数值，不能使用loss[0]
            running_loss += loss.item()
            if i % 200 == 199: # 每200个batch打印一下训练状态
                print('[%d, %5d] loss: %.3f' \
                      % (epoch+1, i+1, running_loss / 200))
                running_loss = 0.0
                
        with torch.no_grad():
            correct, total = 0, 0
            for data in testloader:
                images, labels = data
                va = [[word2ind[key]]]
                va = torch.from_numpy(np.array(va))
                outputs = net(images,va)
                _, predicted = torch.max(outputs, 1)
                #import ipdb;ipdb.set_trace()
                #res = res + [map_[int(predicted[i])] for i in range(len(predicted))]
                total += labels.size(0)
                correct += (predicted == labels).sum()
            if cur_max_acc < float(100 * correct / total):
                print('-------------cur max is ::::----------'+str(float(100 * correct / total)))
                cur_max_acc = float(100 * correct / total)
                torch.save(net, 'charresult/net' + key + '.pkl')  # 保存整个网络
            print('result of epoch ' + str(epoch) + 'accuracy is : %d %%' % (100 * correct / total))

    print('key' + key)
    #net2 = torch.load('net.pkl')
    #prediction = net2(x)
    print('Finished Training')


Net(
  (bilstm): LSTM(100, 100, bidirectional=True)
  (fch): Linear(in_features=200, out_features=100, bias=True)
  (fcv): Linear(in_features=100, out_features=100, bias=True)
  (fcw): Linear(in_features=200, out_features=1, bias=True)
  (fcwp): Linear(in_features=200, out_features=50, bias=True)
  (fcwh): Linear(in_features=200, out_features=50, bias=True)
  (fc1): Linear(in_features=3750, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=4, bias=True)
  (embeddings): Embedding(2876, 100)
)


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   200] loss: 0.337
[1,   400] loss: 0.238
[1,   600] loss: 0.185
[1,   800] loss: 0.233
[1,  1000] loss: 0.221
[1,  1200] loss: 0.227
[1,  1400] loss: 0.217
[1,  1600] loss: 0.231
[1,  1800] loss: 0.252
[1,  2000] loss: 0.232
-------------cur max is ::::----------96.0
result of epoch 0accuracy is : 96 %


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[2,   200] loss: 0.159
[2,   400] loss: 0.175
[2,   600] loss: 0.211
[2,   800] loss: 0.192
[2,  1000] loss: 0.223
[2,  1200] loss: 0.225
[2,  1400] loss: 0.204
[2,  1600] loss: 0.171
[2,  1800] loss: 0.222
[2,  2000] loss: 0.186
result of epoch 1accuracy is : 96 %
[3,   200] loss: 0.191
[3,   400] loss: 0.182
[3,   600] loss: 0.198
[3,   800] loss: 0.179
[3,  1000] loss: 0.187
[3,  1200] loss: 0.171
[3,  1400] loss: 0.164
[3,  1600] loss: 0.227
[3,  1800] loss: 0.141
[3,  2000] loss: 0.203
result of epoch 2accuracy is : 96 %
[4,   200] loss: 0.190
[4,   400] loss: 0.191
[4,   600] loss: 0.165
[4,   800] loss: 0.177
[4,  1000] loss: 0.148
[4,  1200] loss: 0.163
[4,  1400] loss: 0.180
[4,  1600] loss: 0.180
[4,  1800] loss: 0.229
[4,  2000] loss: 0.153
result of epoch 3accuracy is : 96 %
[5,   200] loss: 0.144
[5,   400] loss: 0.167
[5,   600] loss: 0.177
[5,   800] loss: 0.193
[5,  1000] loss: 0.162
[5,  1200] loss: 0.176
[5,  1400] loss: 0.156
[5,  1600] loss: 0.180
[5,  1800] loss: 0

...

[8,   800] loss: 0.135
[8,  1000] loss: 0.137
[8,  1200] loss: 0.164
[8,  1400] loss: 0.159
[8,  1600] loss: 0.130
[8,  1800] loss: 0.134
[8,  2000] loss: 0.189
result of epoch 7accuracy is : 97 %
[9,   200] loss: 0.157
[9,   400] loss: 0.143
[9,   600] loss: 0.137
[9,   800] loss: 0.157
[9,  1000] loss: 0.130
[9,  1200] loss: 0.144
[9,  1400] loss: 0.115
[9,  1600] loss: 0.155
[9,  1800] loss: 0.152
[9,  2000] loss: 0.145
result of epoch 8accuracy is : 97 %
[10,   200] loss: 0.118
[10,   400] loss: 0.129
[10,   600] loss: 0.168
[10,   800] loss: 0.129
[10,  1000] loss: 0.165
[10,  1200] loss: 0.114
[10,  1400] loss: 0.159
[10,  1600] loss: 0.126
[10,  1800] loss: 0.157
[10,  2000] loss: 0.111
result of epoch 9accuracy is : 97 %
[11,   200] loss: 0.133
[11,   400] loss: 0.127
[11,   600] loss: 0.134
[11,   800] loss: 0.134
[11,  1000] loss: 0.125
[11,  1200] loss: 0.141
[11,  1400] loss: 0.129
[11,  1600] loss: 0.138
[11,  1800] loss: 0.135
[11,  2000] loss: 0.136
result of epoch 10acc

[15,  2000] loss: 0.365
result of epoch 14accuracy is : 78 %
[16,   200] loss: 0.335
[16,   400] loss: 0.347
[16,   600] loss: 0.375
[16,   800] loss: 0.337
[16,  1000] loss: 0.341
[16,  1200] loss: 0.310
[16,  1400] loss: 0.332
[16,  1600] loss: 0.346
[16,  1800] loss: 0.398
[16,  2000] loss: 0.330
result of epoch 15accuracy is : 79 %
[17,   200] loss: 0.377
[17,   400] loss: 0.344
[17,   600] loss: 0.341
[17,   800] loss: 0.288
[17,  1000] loss: 0.313
[17,  1200] loss: 0.337
[17,  1400] loss: 0.311
[17,  1600] loss: 0.347
[17,  1800] loss: 0.337
[17,  2000] loss: 0.359
result of epoch 16accuracy is : 79 %
[18,   200] loss: 0.353
[18,   400] loss: 0.303
[18,   600] loss: 0.336
[18,   800] loss: 0.334
[18,  1000] loss: 0.318
[18,  1200] loss: 0.320
[18,  1400] loss: 0.291
[18,  1600] loss: 0.305
[18,  1800] loss: 0.344
[18,  2000] loss: 0.346
result of epoch 17accuracy is : 80 %
[19,   200] loss: 0.388
[19,   400] loss: 0.331
[19,   600] loss: 0.298
[19,   800] loss: 0.260
[19,  1000] 

[1,   400] loss: 0.231
[1,   600] loss: 0.223
[1,   800] loss: 0.212
[1,  1000] loss: 0.229
[1,  1200] loss: 0.184
[1,  1400] loss: 0.164
[1,  1600] loss: 0.166
[1,  1800] loss: 0.169
[1,  2000] loss: 0.148
-------------cur max is ::::----------92.0
result of epoch 0accuracy is : 92 %
[2,   200] loss: 0.169
[2,   400] loss: 0.161
[2,   600] loss: 0.163
[2,   800] loss: 0.155
[2,  1000] loss: 0.145
[2,  1200] loss: 0.159
[2,  1400] loss: 0.172
[2,  1600] loss: 0.139
[2,  1800] loss: 0.167
[2,  2000] loss: 0.127
result of epoch 1accuracy is : 92 %
[3,   200] loss: 0.145
[3,   400] loss: 0.141
[3,   600] loss: 0.147
[3,   800] loss: 0.135
[3,  1000] loss: 0.163
[3,  1200] loss: 0.156
[3,  1400] loss: 0.156
[3,  1600] loss: 0.117
[3,  1800] loss: 0.173
[3,  2000] loss: 0.127
result of epoch 2accuracy is : 92 %
[4,   200] loss: 0.168
[4,   400] loss: 0.140
[4,   600] loss: 0.111
[4,   800] loss: 0.133
[4,  1000] loss: 0.134
[4,  1200] loss: 0.155
[4,  1400] loss: 0.133
[4,  1600] loss: 0.12

[8,  1400] loss: 0.099
[8,  1600] loss: 0.103
[8,  1800] loss: 0.101
[8,  2000] loss: 0.082
result of epoch 7accuracy is : 97 %
[9,   200] loss: 0.102
[9,   400] loss: 0.072
[9,   600] loss: 0.089
[9,   800] loss: 0.069
[9,  1000] loss: 0.067
[9,  1200] loss: 0.102
[9,  1400] loss: 0.102
[9,  1600] loss: 0.094
[9,  1800] loss: 0.058
[9,  2000] loss: 0.092
result of epoch 8accuracy is : 96 %
[10,   200] loss: 0.090
[10,   400] loss: 0.059
[10,   600] loss: 0.077
[10,   800] loss: 0.072
[10,  1000] loss: 0.082
[10,  1200] loss: 0.084
[10,  1400] loss: 0.101
[10,  1600] loss: 0.065
[10,  1800] loss: 0.074
[10,  2000] loss: 0.091
-------------cur max is ::::----------98.0
result of epoch 9accuracy is : 98 %
[11,   200] loss: 0.058
[11,   400] loss: 0.068
[11,   600] loss: 0.100
[11,   800] loss: 0.065
[11,  1000] loss: 0.063
[11,  1200] loss: 0.061
[11,  1400] loss: 0.079
[11,  1600] loss: 0.078
[11,  1800] loss: 0.093
[11,  2000] loss: 0.112
result of epoch 10accuracy is : 97 %
[12,   200

[16,   400] loss: 0.078
[16,   600] loss: 0.086
[16,   800] loss: 0.084
[16,  1000] loss: 0.052
[16,  1200] loss: 0.045
[16,  1400] loss: 0.063
[16,  1600] loss: 0.056
[16,  1800] loss: 0.073
[16,  2000] loss: 0.068
result of epoch 15accuracy is : 92 %
[17,   200] loss: 0.089
[17,   400] loss: 0.075
[17,   600] loss: 0.071
[17,   800] loss: 0.068
[17,  1000] loss: 0.070
[17,  1200] loss: 0.050
[17,  1400] loss: 0.046
[17,  1600] loss: 0.060
[17,  1800] loss: 0.050
[17,  2000] loss: 0.062
result of epoch 16accuracy is : 92 %
[18,   200] loss: 0.053
[18,   400] loss: 0.059
[18,   600] loss: 0.067
[18,   800] loss: 0.054
[18,  1000] loss: 0.068
[18,  1200] loss: 0.067
[18,  1400] loss: 0.068
[18,  1600] loss: 0.056
[18,  1800] loss: 0.044
[18,  2000] loss: 0.061
result of epoch 17accuracy is : 92 %
[19,   200] loss: 0.052
[19,   400] loss: 0.040
[19,   600] loss: 0.073
[19,   800] loss: 0.058
[19,  1000] loss: 0.044
[19,  1200] loss: 0.055
[19,  1400] loss: 0.071
[19,  1600] loss: 0.055
[

In [31]:
import torch.nn as nn
import torch.nn.functional as F
result_set = {}
testset = han_x_test
testloader = torch.utils.data.DataLoader(
                    testset, 
                    batch_size=4,
                    shuffle=False, 
                    num_workers=2)
for key in sub_set:
    print(key)
    res = []
    with torch.no_grad():
        correct, total = 0, 0
        for data in testloader:
            images = data
            va = [[word2ind[key]]]
            va = torch.from_numpy(np.array(va))
            net = torch.load('charresult/net' + key + '.pkl')
            outputs = net(images,va)
            _, predicted = torch.max(outputs, 1)
            res = res + [map_[int(predicted[i])] for i in range(len(predicted))]
    result_set[key] = res


价格


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


配置
操控
舒适性
油耗
动力
内饰
安全性
空间
外观


In [32]:
data = pd.read_csv('../../data/df_sen_sub/train.csv')
uni_sub = data['subject'].unique()
for key in uni_sub:
    print(key + str(len(data[data['subject'] == key])))
    ttt = data[data['subject'] == key]
    print(len(ttt[ttt['sentiment_value'] == 0]))
    print(len(ttt[ttt['sentiment_value'] == -1]))
    print(len(ttt[ttt['sentiment_value'] == 1]))

价格1273
1014
145
114
配置853
579
154
120
操控1036
606
124
306
舒适性931
564
256
111
油耗1082
793
138
151
动力2732
1970
378
384
内饰536
271
150
115
安全性573
380
93
100
空间442
221
67
154
外观489
263
111
115


In [33]:
real_pred = {}
content_ids = []
subjects = []
sentiment_values = []
sentiment_words = []
contents = []
#content_id,subject,sentiment_value,sentiment_word
for i in range(len(test_data)):
    content_id = test_data.loc[i, 'content_id']
    flag = 0
    for key in result_set:
        if result_set[key][i] != '2':
            flag = 1
            content_ids.append(str(content_id))
            subjects.append(key)
            sentiment_values.append(int(result_set[key][i]))
            sentiment_words.append("")
            contents.append(str(test_data.loc[i, 'content']))
    if flag == 0:
        content_ids.append(str(content_id))
        subjects.append('动力')
        sentiment_values.append(0)
        sentiment_words.append("")   
        contents.append(str(test_data.loc[i, 'content']))
real_pred = {"content_id":content_ids, "subject":subjects, "sentiment_value":sentiment_values, "sentiment_word":sentiment_words}
real_df = pd.DataFrame(real_pred)
real_df.to_csv('charresult/final_result++.csv', encoding = "utf-8", index = False)

In [34]:
len(real_df), len(test_data)

(2573, 2364)

In [35]:
real_pred['content'] = contents

dtt = pd.DataFrame(real_pred)
dtt.to_csv('charresult/see_final_result++.csv', encoding = "utf-8", index = False)

In [36]:
len(dtt[dtt['sentiment_value'] == -1])

77

In [38]:
dtt[dtt['sentiment_value'] == 1]

,content_id,subject,sentiment_value,sentiment_word,content
46,M3Hqx1Un8ygzFTtl,动力,1,,我有一个朋友是玩越野的，一直不待见城市SUV的四驱性能，直到有一次一辆森林人跟着他们一起出去...
67,sKPpYiV2bXWEjTLc,空间,1,,q3呗。森林人倒是比q3空间大多了。
87,9Jt0O5d3s6wz1lkf,舒适性,1,,腰不好还是别开车了，森林人坐着不舒服，这个级别就没有适合你的车了
102,SEFvD0TOQgnlJsWK,操控,1,,我有一台老x1，2.0进口的，操控动力都挺不错的，新x1就算了，减配挺严重的，4s店都这么说。
108,WZmFIHJLqT3O1BVk,空间,1,,现在大家有了孩子都买SUV车型，确实，后备箱，空间很大，利于平时家庭使用。
160,zXPLFe5Sqvc06RuG,空间,1,,我个人觉得不是空间问题。只是更喜欢森林人外观。xv不太喜欢。但是价钱上面很是喜欢……我对空间...
188,H6CfVzu4mLq0JkR7,操控,1,,觉着不错！森林人是我的第四辆车！除了内饰看着一般外其他都不错！我也是试驾了所有同类别车后决定...
199,8oeBjF7TwlPyLvcY,舒适性,1,,总之除了森林人，其他车都比森林人舒适
207,qopu4EsiAz9KU7MC,空间,1,,对后排空间要求不高一定是新款XV，新平台不一样
225,7duqegOxLs2Vo8XW,舒适性,1,,想后排舒适就顶配奇骏吧，我就2.0后程超车费劲，那些说2.0也能跑180的滚粗，我也跑过18...
